In [1]:
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')

import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
import re

In [2]:
pd.set_option('display.max_column', None)
train_input = pd.read_csv('train.csv')
test_input = pd.read_csv('test.csv')
print "train_input shape is: " + str(np.shape(train_input))
print "test_input shape is: " + str(np.shape(test_input))

train_input shape is: (4743, 2)
test_input shape is: (1701, 2)


In [3]:
print train_input.head(10)

            handle                                              tweet
0   HillaryClinton  The question in this election: Who can put the...
1   HillaryClinton  Last night, Donald Trump said not paying taxes...
2   HillaryClinton  If we stand together, there's nothing we can't...
3   HillaryClinton  Both candidates were asked about how they'd co...
4  realDonaldTrump  Join me for a 3pm rally - tomorrow at the Mid-...
5   HillaryClinton  When Donald Trump goes low...register to vote:...
6   HillaryClinton  3) Has Trump offered a single proposal to redu...
7   HillaryClinton  The election is just weeks away. Check if you'...
8  realDonaldTrump  Hillary Clinton's Campaign Continues To Make F...
9  realDonaldTrump  'CNBC, Time magazine online polls say Donald T...


In [4]:
def label_tweet(input_set):
    handle = input_set['handle']
    # put it into an array named label, where 0 represents HillaryClinton, 1 represents readDonaldTrump
    label = []
    for i in range(len(handle)):
        if handle[i] == "HillaryClinton":
            label.append(0)
        if handle[i] == "realDonaldTrump":
            label.append(1)
    label = np.asarray(label)
    return label

train_label = label_tweet(train_input)
print train_label[:10]

[0 0 0 0 1 0 0 0 1 1]


In [5]:
train_corpus = train_input['tweet'].as_matrix()
print np.shape(train_corpus)
print train_corpus[:10]
test_corpus = test_input['tweet'].as_matrix()
print np.shape(train_corpus)

(4743,)
[ 'The question in this election: Who can put the plans into action that will make your life better? https://t.co/XreEY9OicG'
 'Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic. https://t.co/t0xmBfj7zF'
 "If we stand together, there's nothing we can't do. \n\nMake sure you're ready to vote: https://t.co/tTgeqxNqYm https://t.co/Q3Ymbb7UNy"
 "Both candidates were asked about how they'd confront racial injustice. Only one had a real answer. https://t.co/sjnEokckis"
 'Join me for a 3pm rally - tomorrow at the Mid-America Center in Council Bluffs, Iowa! Tickets:\xe2\x80\xa6 https://t.co/dfzsbICiXc'
 'When Donald Trump goes low...register to vote: https://t.co/tTgeqxNqYm https://t.co/DXz9dEwsZS'
 '3) Has Trump offered a single proposal to reduce the friction of starting a business. @HillaryClinton has https://t.co/OhFAFEFsUq'
 "The election is just weeks away. Check if you're registered to vote at https://t.co/HcMAh8ljR0, only takes a f

In [31]:
# Load the stopwords
stop_words = stopwords.words('english')
# 'https' seems useless, so I add it to stop_words
stop_words.append(u'https')
print stop_words

[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u

## Tokenize

In [27]:
# Tokenize
def tokenization(text):
    tokens=[]
    for word in nltk.word_tokenize(text.decode('utf-8')):
        # skip all the websites, punctuations, pure digits
        if not re.match('[//]', word) and re.search('[a-zA-Z]', word) and word.lower() not in stop_words:
            tokens.append(word.lower())
    return tokens

# Tokenize training set
train_corpus_tokenized = []
for i in train_corpus:
    train_corpus_tokenized.append(' '.join(tokenization(i)))
    
print len(train_corpus_tokenized)
print train_corpus_tokenized[:10]

# Tokenize testing set
test_corpus_tokenized = []
for i in test_corpus:
    test_corpus_tokenized.append(' '.join(tokenization(i)))
    
# print test_corpus_tokenized[:10]

4743
[u'question election put plans action make life better', u'last night donald trump said paying taxes smart know call unpatriotic', u"stand together 's nothing ca n't make sure 're ready vote", u"candidates asked 'd confront racial injustice one real answer", u'join 3pm rally tomorrow mid-america center council bluffs iowa tickets', u'donald trump goes low register vote', u'trump offered single proposal reduce friction starting business hillaryclinton', u"election weeks away check 're registered vote takes cl\u2026", u"hillary clinton 's campaign continues make false claims foundation disclosure", u"'cnbc time magazine online polls say donald trump first presidential debate via washtimes maga"]


In [30]:
# import collections
# cnt = collections.Counter()
# for i in range(len(train_corpus_tokenized)):
#     for word in train_corpus_tokenized[i]:
#         cnt[word] += 1
    
# print cnt


## Word embedding

In [29]:
# vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
# train_input_transform = vocab_processor.fit_transform(train_input['tweet'])
train = []
for i in range(len(train_corpus_tokenized)):
    train.append(tf.compat.as_str(train_corpus_tokenized[i]).split())
print len(train)
print train[:20]

vocabulary_size = 50000

4743
[['question', 'election', 'put', 'plans', 'action', 'make', 'life', 'better'], ['last', 'night', 'donald', 'trump', 'said', 'paying', 'taxes', 'smart', 'know', 'call', 'unpatriotic'], ['stand', 'together', "'s", 'nothing', 'ca', "n't", 'make', 'sure', "'re", 'ready', 'vote'], ['candidates', 'asked', "'d", 'confront', 'racial', 'injustice', 'one', 'real', 'answer'], ['join', '3pm', 'rally', 'tomorrow', 'mid-america', 'center', 'council', 'bluffs', 'iowa', 'tickets'], ['donald', 'trump', 'goes', 'low', 'register', 'vote'], ['trump', 'offered', 'single', 'proposal', 'reduce', 'friction', 'starting', 'business', 'hillaryclinton'], ['election', 'weeks', 'away', 'check', "'re", 'registered', 'vote', 'takes', 'cl\xe2\x80\xa6'], ['hillary', 'clinton', "'s", 'campaign', 'continues', 'make', 'false', 'claims', 'foundation', 'disclosure'], ["'cnbc", 'time', 'magazine', 'online', 'polls', 'say', 'donald', 'trump', 'first', 'presidential', 'debate', 'via', 'washtimes', 'maga'], ['donald', 'tru

In [33]:
# def build_dataset(words, n_words):
#     """Process raw inputs into a dataset."""
#     count = [['UNK', -1]]
#     count.extend(collections.Counter(words).most_common(n_words - 1))
#     dictionary = dict()
#     for word, _ in count:
#         dictionary[word] = len(dictionary)
#     data = list()
#     unk_count = 0
#     for word in words:
#         index = dictionary.get(word, 0)
#         if index == 0:  # dictionary['UNK']
#             unk_count += 1
#         data.append(index)
#     count[0][1] = unk_count
#     reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
#     return data, count, dictionary, reversed_dictionary

# data, count, dictionary, reverse_dictionary = build_dataset(train, vocabulary_size)

In [39]:
cnt = collections.Counter()
for i in range(len(train)):
    for word in train[i]:
        cnt[word] += 1

print len(cnt)
print cnt

8507
Counter({'trump': 965, "'s": 768, 'hillary': 594, 'thank': 414, 'great': 391, "n't": 371, 'donald': 355, 'realdonaldtrump': 339, 'people': 303, 'america': 302, 'president': 270, 'make': 268, 'amp': 266, 'trump2016': 254, 'clinton': 239, '\xe2\x80\x94hillary': 232, 'one': 207, 'new': 205, 'get': 189, 'would': 181, 'makeamericagreatagain': 176, 'vote': 169, 'us': 168, 'let': 165, 'like': 158, 'cruz': 147, 'today': 147, 'crooked': 142, 'country': 134, 'time': 133, 'women': 128, 'potus': 127, 'many': 124, 'never': 122, 'campaign': 120, 'big': 119, 'going': 116, 'need': 115, 'back': 114, 'tonight': 113, 'american': 112, 'watch': 109, 'join': 107, 'last': 103, 'want': 102, 'americans': 102, 'said': 100, 'good': 100, 'ted': 100, 'go': 99, 'day': 99, 'families': 98, 'first': 97, 'much': 95, 'win': 95, 'hillaryclinton': 93, 'see': 91, 'together': 90, 'know': 89, 'every': 87, 'got': 87, 'ca': 86, 'love': 85, 'job': 84, 'jobs': 83, 'support': 81, 'years': 81, 'cnn': 81, 'says': 80, 'bad': 79